# OllaBridge — Google Colab Demo (Client + Remote Connection)

This notebook shows **two common Colab patterns**:

1. **Colab as a client** connecting to your **local OllaBridge gateway** (most common).
2. **Colab hosting OllaBridge** and optionally exposing it via a **public tunnel** (for demos).

> Colab can’t reach your laptop’s `localhost` directly.  
> To connect Colab → your local gateway, you must expose your gateway on a reachable URL (VPN, reverse proxy, or a tunnel).

**Last updated:** 2026-01-05

## 0) Install dependencies (Colab)

This installs:
- `openai` (official SDK)
- `requests` (health checks)

In [1]:
!python -m pip -q install -U openai requests

## 1) Option A (recommended): Colab → Your Local Gateway via a Tunnel

### Step A1 — Start OllaBridge locally (on your machine)

In a terminal on your machine:

```bash
.venv/bin/python3 -m ollabridge.cli.main start --host 0.0.0.0 --port 11435 --reload --log-level info
```

### Step A2 — Expose it to the internet (quick demo tunnel)

Pick **one**:
- Cloudflare Quick Tunnel (`cloudflared`) — simplest for demos
- ngrok — popular, but requires an account/token

**Enterprise note:** For production, prefer VPN / private networking + TLS + allowlists.

### Step A3 — Use the public URL in Colab

Once you have:
- `PUBLIC_BASE_URL` like `https://<random>.trycloudflare.com/v1`
- Your existing `OLLAS_API_KEY` from the banner

Paste them below and run.

In [2]:
import os

# 🔧 Paste your public base URL here (must include /v1)
PUBLIC_BASE_URL = os.getenv("PUBLIC_BASE_URL", "https://YOUR_TUNNEL_DOMAIN/v1")

# 🔐 Paste the key shown by your local gateway banner
OLLAS_API_KEY = os.getenv("OLLAS_API_KEY", "sk-ollabridge-REPLACE_ME")

# Model name as shown in the banner (example: deepseek-r1)
MODEL = os.getenv("OLLAS_MODEL", "deepseek-r1")

print("PUBLIC_BASE_URL:", PUBLIC_BASE_URL)
print("Key set:", OLLAS_API_KEY.startswith("sk-ollabridge-"))
print("MODEL:", MODEL)

PUBLIC_BASE_URL: https://YOUR_TUNNEL_DOMAIN/v1
Key set: True
MODEL: deepseek-r1


In [3]:
import requests
from urllib.parse import urlparse

def gateway_health(base_url: str, timeout_s: int = 10):
    """Return (ok: bool, payload_or_error: object)."""
    try:
        health_url = base_url.replace("/v1", "") + "/health"
        r = requests.get(health_url, timeout=timeout_s)
        r.raise_for_status()
        return True, r.json()
    except Exception as e:
        return False, str(e)

def require_gateway(base_url: str):
    ok, info = gateway_health(base_url, timeout_s=10)
    if ok:
        print("✅ Gateway reachable:", base_url)
        print("Health:", info)
        return True
    print("⚠️ Gateway NOT reachable:", base_url)
    print("Reason:", info)
    print("\nTip: start OllaBridge first, or set the correct base URL / tunnel URL.")
    return False

### Step A4 — Health check through the tunnel

In [4]:
# Health check (safe): won't crash the notebook if the gateway isn't up yet.
health_url = PUBLIC_BASE_URL.replace("/v1", "") + "/health"
ok, info = gateway_health(PUBLIC_BASE_URL, timeout_s=20)
if ok:
    info
else:
    print("Skipping health check because gateway is not reachable.")
    print("Reason:", info)

Skipping health check because gateway is not reachable.
Reason: HTTPSConnectionPool(host='your_tunnel_domain', port=443): Max retries exceeded with url: /health (Caused by NameResolutionError("HTTPSConnection(host='your_tunnel_domain', port=443): Failed to resolve 'your_tunnel_domain' ([Errno -2] Name or service not known)"))


### Step A5 — Call the gateway with the OpenAI SDK

In [5]:
if not require_gateway(PUBLIC_BASE_URL):
    print('⏭️ Skipping this cell (gateway not reachable).')
else:
    from openai import OpenAI

    client = OpenAI(base_url=PUBLIC_BASE_URL, api_key=OLLAS_API_KEY)

    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system","content":"You are a helpful assistant."},
            {"role":"user","content":"In 5 lines, explain what OllaBridge does and why it helps teams."},
        ],
        temperature=0.2,
    )

    print(resp.choices[0].message.content)

⚠️ Gateway NOT reachable: https://YOUR_TUNNEL_DOMAIN/v1
Reason: HTTPSConnectionPool(host='your_tunnel_domain', port=443): Max retries exceeded with url: /health (Caused by NameResolutionError("HTTPSConnection(host='your_tunnel_domain', port=443): Failed to resolve 'your_tunnel_domain' ([Errno -2] Name or service not known)"))

Tip: start OllaBridge first, or set the correct base URL / tunnel URL.
⏭️ Skipping this cell (gateway not reachable).


## 2) Option B: Run OllaBridge **inside Colab** (advanced / demo-only)

This is useful if:
- you want a fully Colab-hosted demo
- your model/workers can run in Colab (GPU availability varies)

> **Note:** Running local-model stacks (e.g., Ollama) in Colab can be non-trivial and may require extra setup.
> This section focuses on the **gateway mechanics** + exposure pattern.

### Step B1 — Install OllaBridge in Colab

In [6]:
# If OllaBridge is published on PyPI, this should work:
# !pip install -U ollabridge
#
# If you instead use a git install, you can do something like:
# !pip install -U "git+https://github.com/<org>/ollabridge.git"
#
# For this notebook we keep it as a placeholder command:
!echo "Install OllaBridge here (pip install -U ollabridge)"

Install OllaBridge here (pip install -U ollabridge)


### Step B2 — Start the gateway in the background (Colab)

If `ollabridge` is installed and available, a command like this starts the server:

```bash
python -m ollabridge.cli.main start --host 0.0.0.0 --port 11435 --log-level info
```

In Colab, you typically run long-lived servers using `subprocess.Popen(...)`.

In [7]:
import subprocess, sys, textwrap, shlex, time

print("This cell shows a typical pattern for starting a server in Colab.")
print("Uncomment and adjust once ollabridge is installed and configured.")

# cmd = "python -m ollabridge.cli.main start --host 0.0.0.0 --port 11435 --log-level info"
# proc = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# time.sleep(2)
# print("Server started (pid)", proc.pid)

This cell shows a typical pattern for starting a server in Colab.
Uncomment and adjust once ollabridge is installed and configured.


### Step B3 — Call the Colab-hosted gateway from *within Colab*

If you run the gateway in the same Colab runtime, your base URL is:

- `http://127.0.0.1:11435/v1`

(Still uses the same OpenAI-compatible client pattern.)

In [8]:
# If the gateway is running in this same Colab runtime:
COLAB_BASE_URL = "http://127.0.0.1:11435/v1"
print("COLAB_BASE_URL:", COLAB_BASE_URL)

COLAB_BASE_URL: http://127.0.0.1:11435/v1


## 3) (Optional) Expose a Colab-hosted gateway publicly with Cloudflare Quick Tunnel

If you want to share a live demo link, you can create a tunnel from Colab → internet.

> This is a **demo convenience**, not a production pattern.

### Step C1 — Install cloudflared

In [9]:
!echo "Optional: install cloudflared here (pip install -U cloudflared)"

Optional: install cloudflared here (pip install -U cloudflared)


### Step C2 — Start the tunnel

This creates a public URL that forwards to `http://127.0.0.1:11435`.

The output contains a `trycloudflare.com` URL. Use that as your public base URL:
- `https://<random>.trycloudflare.com/v1`

In [10]:
import subprocess, shlex, time, re, os, sys

print("Starting Cloudflare Quick Tunnel to http://127.0.0.1:11435 ...")
print("If the cell hangs, scroll output; the public URL should appear.")

# Start tunnel (will keep running)
# proc = subprocess.Popen(
#     shlex.split("cloudflared tunnel --url http://127.0.0.1:11435"),
#     stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
# )

# Example snippet parser (uncomment when running for real):
# public_url = None
# for _ in range(80):
#     line = proc.stdout.readline().strip()
#     if line:
#         print(line)
#     m = re.search(r"https://[\w\-]+\.trycloudflare\.com", line)
#     if m:
#         public_url = m.group(0)
#         break
# if public_url:
#     print("\n✅ Tunnel URL:", public_url)
#     print("Set PUBLIC_BASE_URL =", public_url + "/v1")

Starting Cloudflare Quick Tunnel to http://127.0.0.1:11435 ...
If the cell hangs, scroll output; the public URL should appear.


## 4) Enterprise checklist (when you share a gateway)

- Put the gateway behind **TLS**
- Keep the API key in a **secret store** (not notebooks)
- Use **allowlists** / IP restrictions for demo links
- Rotate keys and log access
- Disable `--reload` outside dev

If you share your demo with others, create a separate key dedicated to the demo and rotate it after the session.